This was used for testing

In [1]:
from strategies.moving_average_cross import MovingAverageCrossStrategy
from backtest import Backtest
from metrics import PerformanceMetrics
from itertools import product
import pandas as pd
import numpy as np
import itertools
from joblib import Parallel, delayed
from tqdm import tqdm

data = pd.read_csv("data/msft.csv", index_col="Date", parse_dates=True)
data = data.loc['2020-01-01':'2024-12-31']

In [ ]:
trend_thresholds = range(5, 30, 5)
adx_thresholds = range(10, 35, 5)
short_windows = range(5, 20, 5)
long_windows = range(30, 100, 10)
stop_losses = [0.01, 0.02, 0.03, 0.04]
take_profits = [0.02, 0.05, 0.07, 0.1]

param_grid = list(itertools.product(
    trend_thresholds,
    adx_thresholds,
    short_windows,
    long_windows,
    stop_losses,
    take_profits
))


def evaluate_params(params, data):
    trend_thr, adx_thr, short_ma, long_ma, sl_pct, tp_pct = params

    if short_ma >= long_ma:
        return None  # skip invalid combo

    try:
        strategy = MovingAverageCrossStrategy(
            short_ma, 
            long_ma,
            adx_threshold=adx_thr,
            trend_direction_threshold=trend_thr,
            stop_loss_pct=sl_pct,
            take_profit_pct=tp_pct
        )

        backtest = Backtest(data, strategy)
        results = backtest.run()
        metrics = PerformanceMetrics(results, trades_df=backtest.trade_history_df)

        sharpe = metrics.calculate_sharpe_ratio()
        drawdown = metrics.calculate_max_drawdown()
        returns = metrics.calculate_total_return()

        if drawdown <= 1:
            drawdown = 9999
        score = (2 * sharpe) + (1 * returns) - (3 * drawdown)

        return {
            'params': params,
            'score': score,
        }

    except Exception as e:
        print(f"Error for params {params}: {e}")
        return None

In [ ]:

n_jobs = -1 

results = Parallel(n_jobs=n_jobs)(
    delayed(evaluate_params)(params, data) for params in tqdm(param_grid)
)

# Filter out any None results
results = [r for r in results if r is not None]

# Sort by score
results_df = pd.DataFrame(results)
results_df = results_df.sort_values(by='score', ascending=False)

100%|██████████| 8400/8400 [01:22<00:00, 102.22it/s]


In [4]:
results_df

,params,score
320,"(5, 10, 15, 90, 0.01, 0.02)",-29994.279290
332,"(5, 10, 15, 90, 0.04, 0.02)",-29994.407031
222,"(5, 10, 10, 90, 0.04, 0.07)",-29994.416449
334,"(5, 10, 15, 90, 0.04, 0.07)",-29994.437465
220,"(5, 10, 10, 90, 0.04, 0.02)",-29994.535991
...,...,...
5764,"(20, 20, 5, 60, 0.02, 0.02)",-29999.692222
3941,"(15, 15, 15, 40, 0.02, 0.05)",-29999.728359
3829,"(15, 15, 10, 40, 0.02, 0.05)",-29999.777062
5380,"(20, 15, 5, 30, 0.02, 0.02)",-29999.815458


In [ ]:
a = [1,2,3,4,5,6]
mid = len(a)//2

[4, 5, 6]